In [1]:
%matplotlib tk

# Directory manipulation for relative notebook imports
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

# Relative import for relevant methods
from Visualization.plot_util import *
from Visualization.projection_plots import *
from Data.data_load_util import *
from Simulation.projections_util import *
from Models.Neat.neat_util import *
from Simulation.projections import *

import pickle

In [2]:
# Loading the datasets 
zips_df, state_df, pos_df = make_dataset(granularity='both', remove_outliers=False, load_dir_prefix='../Data/')

data_manager = DataManager(zips_df, fields=['Median_income', 'carbon_offset_kg_per_panel', 'yearly_sunlight_kwh_kw_threshold_avg', 'panel_utilization', 'black_prop'])


In [6]:
#test the milf vs regular model
with open(dir1 + "\\Models\\Neat\\models\\NEAT_model_lexicase_base_weights.pkl", 'rb') as f:
    network = pickle.load(f)

neat_proj = create_neat_proj(data_manager, 100000, NeatModel(network), create_paper_objectives())


In [3]:

with open("test_milp_placement.pkl", 'rb') as f:
    milp_placement = pickle.load(f)
# print (milp_placement)
milp_proj = create_projection_from_panel_assignment(zips_df, milp_placement, create_paper_objectives())

In [5]:

num_panels = 10000000
#get the NEAT projection
with open(dir1 + "\\Models\\Neat\\models\\05-30-25\\NEAT_model_lexicase_weighted_3.pkl", 'rb') as f:
    network = pickle.load(f)

#2 million panels
neat_proj_path_2m = "../Simulation/Projection_Saves/projections_neat_2m.pkl"
neat_proj_new_2m = create_neat_proj(data_manager, 2000000, NeatModel(network), create_paper_objectives(), save=neat_proj_path_2m, load=neat_proj_path_2m)


#10 million panels
neat_proj_path_10m = "../Simulation/Projection_Saves/projections_neat_10m.pkl"
neat_proj_new_10m = create_neat_proj(data_manager, 10000000, NeatModel(network), create_paper_objectives(), save=neat_proj_path_10m, load=neat_proj_path_10m)

#original model
# with open(dir1+"/Models/Neat/models/01-09-25/NEAT_model2M_lexicase.pkl", 'rb') as f:
#     network = pickle.load(f)


# proj_path2 = "../Simulation/Projection_Saves/projections_neat_1m.pkl"
# neat_proj_original = create_neat_proj(data_manager, num_panels, NeatModel(network), create_paper_objectives(), save=proj_path2, load=proj_path2)



Loading from previous calculations...
Loading from previous calculations...


In [6]:
#Get all other projections
proj_path_2m = "../Simulation/Projection_Saves/projections_2m.pkl"
projections_2m = create_projections(zips_df, state_df, n_panels=2000000, save=proj_path_2m, load=proj_path_2m)

proj_path_10m = "../Simulation/Projection_Saves/projections_10m.pkl"
projections_10m = create_projections(zips_df, state_df, n_panels=10000000, save=proj_path_10m, load=proj_path_10m)

Loading from previous simulation...
Loading from previous simulation...


In [7]:
#code to edit a projection
# projections[1] = create_future_estimate_projection(zips_df, state_df, n_panels=num_panels, objectives=create_paper_objectives())
# #save
# with open(proj_path2, 'wb') as dir:
#     pickle.dump(projections, dir, pickle.HIGHEST_PROTOCOL)

In [8]:
compared_projections_2m = [projections_2m[1], projections_2m[2], projections_2m[3]] + [neat_proj_new_2m] #includes future installs, carbon-optimal, energy-optimal, and neat
compared_projections_10m = [projections_10m[1], projections_10m[2], projections_10m[3]] + [neat_proj_new_10m] #includes future installs, carbon-optimal, energy-optimal, and neat

In [9]:
#color palettes

# methods_colorpalette = sns.color_palette("colorblind")[0:3] + sns.color_palette("colorblind")[4:] #skip the other orange color

# methods_colorpalette = sns.color_palette("colorblind")[0:1] + sns.color_palette("Set2")[0:2] + sns.color_palette("Set2")[3:4]

# methods_colorpalette = ["#0000a2","#bc272d", "#e9c716","#50ad9f"]

methods_colorpalette = ["#4a2377","#8cc5e3","#0d7d87", "#f55f74"]

# methods_colorpalette = ["#d31f11","#f47a00", "#62c8d3","#007191"]

# methods_colorpalette = ["#d31f11","#00a303ba", "#a14ca26a","#077A9A"]

In [10]:
#fig 3
plot_projections(compared_projections_2m, "Carbon Offset", color_palette=methods_colorpalette, interval=100000)
plot_projections(compared_projections_2m, "Energy Potential", color_palette=methods_colorpalette, interval=100000)

In [ ]:
# plot_comparison_ratio(projections_10m[1], neat_proj_new_10m, "base","comp", interval=100000) 

In [ ]:
annual_new_panels = [720000 * 1.09**i for i in range(3)]
for num_panels in [sum(annual_new_panels[:i]) for i in range(1,4)]:
    plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=num_panels, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)
# plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=1000000, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)
# plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=2000000, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)
# plot_bar_comparison_ratio(projections_10m[1], [projections_10m[2], projections_10m[3], neat_proj_new_10m], panel_count=10000000, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5)

In [ ]:
# plot_equity_comparison(neat_proj_new, create_equity_objectives(), panel_counts = [0,100000,1000000], color_palette=sns.color_palette("Greens", 3)) #fig 4

In [ ]:
# palette = sns.color_palette("Set1")
# sns.palplot(palette)

In [11]:
plot_bar_comparison_ratio(projections_2m[1], [projections_2m[2], neat_proj_new_2m, milp_proj], panel_count=num_panels, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)